In [ ]:
import numpy as np
import cv2
import math
from PIL import Image, ImageOps
from matplotlib import pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
import os
import glob

In [ ]:
'''
Makes found boxes by get_boxes() square
'''
def make_square(boxes):
    squares = []
    for box in boxes:
        x,y,w,h = box
        if h > w:
            diff = h-w
            x = math.floor(x - (diff/2))
            w = math.ceil(w + diff)
    
        if w > h:
            diff = w-h
            y = math.floor(y - (diff/2))
            h = math.ceil(h + diff)
            
        squares.append([x,y,w,h])
    return squares

In [ ]:
'''
!!! Test only function !!!
Shows the found squares by drawing them on the image
'''

def draw_squares(im,squares):
    for square in squares:
        x,y,w,h = square
        im2 = cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),2)
        plt.imshow(im2)
    plt.show()

In [ ]:
######################################################################

In [ ]:
'''
Turns Squares around rois into actual images
'''
def get_found_imgs(im,squares):
    found_imgs = []
    for square in squares:
        print("square:",square)

        new_im = im[square[1]:square[1]+square[3], square[0]:square[0]+square[2]]
        img = Image.fromarray(new_im, 'RGB')
        found_imgs.append(img)
    return found_imgs

In [ ]:
'''
Scales all found images to one size
if size == 0: No scaling

''' 
def scale_imgs(imgs,size):
    scaled_imgs = []
    for im in imgs:
        if size == 0:
            new_im = im
        else:
            new_im = ImageOps.fit(im, size, Image.ANTIALIAS)
        scaled_imgs.append(new_im)
    return scaled_imgs

In [ ]:
cor = [30,40,30]


In [ ]:
'''
Saves found images to files in current directory.
'''
def imgs_to_files(imgs,nom, cors, labels=None ):
    count = 0
    labeled_directory = "cnn_data4"
    if not os.path.isdir('./' + labeled_directory):
        os.mkdir(labeled_directory)
    for idx, im in enumerate(imgs):
        cor = '_'.join([str(e) for e in cors[idx]])
        name = labeled_directory + "/canditate_"+nom+ "_"+ str(count) + "_cors_"+ cor + "_isRobot_" + str(labels[count]) + ".png"
        print(name)
        im.save(name)   
        count += 1
    
    return

In [ ]:
###############################################################################

In [ ]:
'''
In case of blobs:
y,x,sigma
All of the below!
'''

In [ ]:
def make_blob_square(blobs):
    squares = []

    for blob in blobs:
        
        y_cen, x_cen, rad = blob
        
        x = x_cen - math.floor(rad)
        y = y_cen - math.floor(rad)
        
        w = rad*2
        h = rad*2
        
        xsiz= 479 - w
        ysiz = 639 - h
        
        if x < 0:
            x = 0
        elif x > xsiz:
            x = xsiz
        if y < 0:
            y = 0
        elif y > ysiz:
            y = ysiz
        square = [int(x),int(y),int(w),int(h)]
        squares.append(square)
    return squares

In [ ]:
def get_roi_imgs_blobs(blobs,im_ori):
    squares = make_blob_square(blobs)
    imgs = get_found_imgs(im_ori,squares)
    
    return imgs, squares

In [ ]:
##########################################################################################

In [ ]:
'''Labeling'''
def make_label_file(labels):
    labels = labels.readlines()
    labels = labels[6:]
    current_im = ''
    current_borders = []
    borders = []
    for label in labels:
        cells = label.split('|')
#         print(cells)
        these_borders = get_coords_from_label(cells)
        
        if cells[1] != current_im:
            current_im = cells[1]
            borders.append(current_borders)
            current_borders = [current_im]
        
        current_borders.append(these_borders)
    
    return borders[1:]

In [ ]:
l = open('labels.txt')
my_labels = make_label_file(l)
my_labels

In [ ]:
def get_coords_from_label(cells):
    if cells[2] == 'not_in_image\n':
        return [0,0,0,0]
    else:
        cent_x = float(cells[8])
        cent_y = float(cells[9])
        w = int(cells[10])
        h = int(cells[11])

        x = cent_x - math.floor(w/2)
        y = cent_y - math.floor(h/2)

        square = [int(x),int(y),int(w),int(h)]
    
    return square

In [ ]:
def check_overlap(List1, List2):  # returns None if rectangles don't intersect
#     print(List1,List2)
    thresh = 0.5
    [x1, y1, w1, h1] = List1  #van robot
    [x2,y2,w2,h2] = List2 #van Blob
    a = [x1 , y1+ h1, x1+w1, y1] # area robot
    b = [x2 , y2+ h2, x2+w2, y2] # area blob
    dx = min(a[2], b[2]) - max(a[0], b[0])  #'xmin ymin xmax ymax'
    dy =  min(a[1], b[1]) - max(a[3], b[3])

    
    if (dx>=0) and (dy>=0):
        opvblob = w2*h2
        intsecDivOpv = dx*dy/opvblob
#         print("intersection:", dx*dy,"/oppervlakte",opvblob,"IOU",intsecDivOpv)
        if intsecDivOpv > thresh:
#             print("good candid")
            return True
#     print("bad candid")
    else:
        print("geen intersect")
    return False


In [ ]:
def check_squares(squares,labels):
    # squares: found by blob detection
    # labels: of the robots
    bool_list = []
    for square in squares:
        is_in_robot = 0
        for label in labels:
#             print(square,label)
            if check_overlap(label, square):
                is_in_robot = 1
        bool_list.append(is_in_robot)
            
    return bool_list
        

In [ ]:
def get_labeled_imgs(labels,size):
    for label in labels:
        img_name = label[0]
#         print(img_name)
        im = cv2.imread("imageset_131/" + img_name)
        bin_dir = 'bin_image2/'
        
        
        bin_img_name = bin_dir + img_name
        print("imgname:",img_name[-13:-10])
        bin_img = cv2.imread(bin_img_name,0)
        
        blobs = blob_doh(bin_img, max_sigma=50, min_sigma=30, threshold=.01, num_sigma=10)
#         print("blobs:",blobs)
        ''' dit is een lijst met blobs'''
        
        found_imgs, squares = get_roi_imgs_blobs(blobs,im)
#         print("squaresMain",squares)
        #Returns list of booleans for found regions
        in_robots = check_squares(squares, label[1:])
        
        imgs = scale_imgs(found_imgs, size)
        imgs_to_files(imgs,img_name[-13:-10],squares,in_robots)
            
    return

In [ ]:
get_labeled_imgs(my_labels,tuple((60,60)))

In [ ]:
leon = "16_02_2018__11_18_08_0019_upper.png"
print(leon[-3:])

### stappenplan

Remove eerste paar lines van labels.txt

dan run labeldingie make_label_file

zorgt dat deze ipynb in map met imagaes staat (en labels.txt ook)

run die shit!


size = ? tuple van pixels
dus bijv (60,60)

get_labeled_imgs(labels,size)